In [3]:
import glob
import os
from os import chdir as cd
from astropy import units as u
from astropy.coordinates import *
#from ciao_contrib.runtool import *
import pandas as pd
import numpy as np
from numpy import sin,cos
from astropy.cosmology import Planck15

In [81]:
cd('/Users/leo/Documents/College/Research/CHANGES_Xray_analysis')

objList = sorted(glob.glob('NGC*.png'))
galaxies = []
for obj in objList:
    galaxies.append(obj.replace('.png',''))
galaxies.insert(0,'NGC891')
galaxies.insert(0,'NGC660')
galaxies.pop(-1)
galaxies.pop(-1)
print(galaxies)

cd('/Users/leo/Documents/College')

['NGC660', 'NGC891', 'NGC2683', 'NGC2992', 'NGC3079', 'NGC3432', 'NGC3448', 'NGC3556', 'NGC3628', 'NGC3877', 'NGC4013', 'NGC4096', 'NGC4157', 'NGC4192', 'NGC4217', 'NGC4244', 'NGC4302', 'NGC4388', 'NGC4438', 'NGC4565', 'NGC4594', 'NGC4631', 'NGC4666', 'NGC5084', 'NGC5297', 'NGC5775', 'NGC5907']


In [7]:
def ellipse(x,y,h,k,a,b,A):
    return ((((x-h)*cos(y)*cos(A)+(y-k)*sin(A))**2)/a**2)+((((x-h)*cos(y)*sin(A)-(y-k)*cos(A))**2)/b**2)

fileList = sorted(glob.glob("*/*/*xray_src.csv"))

a_arcmin = [3.16, 13.5, 9.33, 3.55, 7.94, 6.76, 5.62, 8.71, 14.8, 5.50, 5.25, 6.61, 6.76, 9.77, 5.25, 16.6, 5.50, 5.62, 8.51, 15.9, 8.71, 15.5, 4.57, 9.33, 5.62, 4.17, 12.6]
b_arcmin = [0.98, 2.51, 2.19, 1.10, 1.45, 1.48, 1.78, 2.24, 2.95, 1.29, 1.02, 1.78, 1.35, 2.75, 1.55, 1.91, 1.00, 1.29, 3.16, 2.14, 3.55, 2.69, 1.29, 1.74, 1.26, 1.00, 1.38]
A_north = [41, 24, 41, 22, 179, 42.0, 65, 81, 104, 34, 66, 17, 63, 152, 49, 45, 178, 89, 20, 134, 87, 84, 40, 80, 149, 146, 155]

a_deg = 0.0166667*np.array(a_arcmin)
b_deg = 0.0166667*np.array(b_arcmin)
A = 90+np.array(A_north)

print(a_deg[-2],b_deg[-2],A[-2])

#
hms_ra_c = ['01h43m2.40s', '02h22m33.41s', '08h52m41.35s', '09h45m42.00s', '10h01m57.80s', '10h52m31.13s', '10h54m39.24s', '11h11m30.97s', '11h20m17.01s', '11h46m07.80s', '11h58m31.38s', '12h06m01.13s', '12h11m04.37s', '12h13m48.29s', '12h15m50.90s', '12h17m29.66s', '12h21m42.48s', '12h25m46.75s', '12h27m45.59s', '12h36m20.78s', '12h39m59.43s', '12h42m08.01s', '12h45m08.59s', '13h20m16.92s', '13h46m23.68s', '14h53m57.60s', '15h15m53.77s']
dms_dec_c = ['+13d38m42.2s', '+42d20m56.9s', '+33d25m18.5s', '-14d19m35.0s', '+55d40m47.3s', '+36d37m07.6s', '+54d18m18.8s', '+55d40m26.8s', '+13d35m22.9s', '+47d29m41.2s', '+43d56m47.7s', '+47d28m42.4s', '+50d29m04.8s', '+14d54m01.2s', '+47d05m30.4s', '+37d48m25.6s', '+14d35m53.9s', '+12d39m43.5s', '+13d00m31.8s', '+25d59m15.6s', '-11d37m23.0s', '+32d32m29.4s', '-00d27m42.8s', '-21d49m39.3s', '+43d52m20.5s', '+03d32m40.0s', '+56d19m43.6s']
deg_ra_c = []
deg_dec_c= []

for ra,dec in zip(hms_ra_c,dms_dec_c):
    c = SkyCoord(ra, dec)
    deg_ra_c.append(c.ra.degree)
    deg_dec_c.append(c.dec.degree)

flux_conversion = [ 1.189e-11,  1.098e-11,  1.055E-11,  1.129E-11,  1.206E-11,  1.055E-11,  2.220E-11,  9.125E-12,  9.125E-12,  9.125E-12,  1.099E-11,  1.273E-11,  1.178E-11,  1.924E-11,  1.099E-11,  9.125E-12,  2.220E-11,  1.055E-11,  9.217E-12,  9.217E-12,  1.099E-11,  9.125E-12,  1.129E-11,  1.202E-11,  1.924E-11,  9.217E-12,  1.511E-11]
redshift = [0.002835, 0.001761, 0.001, 0.008, 0.004, 0.002, 0.0045, 0.002, 0.0028, 0.003, 0.0027, 0.0018, 0.0026, -0.0002, 0.0034, 8e-4, 0.0038, 0.008, 3e-4, 0.0041, 0.0036, 0.0021, 0.0051, 0.006, 0.0082, 0.0056, 0.002]

frames = []

for i,file, gal, flux, z in zip(range(27),fileList, galaxies, flux_conversion, redshift):
    df = pd.read_csv(file, index_col=False, usecols=['Unnamed: 0', 'ID', 'RA', 'DEC', 'Sft_Cts', 'Sft_Rate', 'Med_Cts', 'Med_Rate', 'Hrd_Cts', 'Hrd_Rate', 'Brd_Cts', 'Brd_Rate', 'HR1', 'HR2'])
    df['Unnamed: 0'] = gal
    df['RA Center'] = deg_ra_c[i]
    df['DEC Center'] = deg_dec_c[i]
    df['d25'] = a_deg[i]
    df['r25'] = b_deg[i]
    df['Pos Angle'] = A[i]
    df['Sft_Rate'] = df['Sft_Rate']*10**(-3)
    df['Med_Rate'] = df['Med_Rate']*10**(-3)
    df['Hrd_Rate'] = df['Hrd_Rate']*10**(-3)
    df['Brd_Rate'] = df['Brd_Rate']*10**(-3)
    df['Exposure Time'] = df['Brd_Cts']/df['Brd_Rate']
    r = np.abs(Distance(z=z, cosmology=Planck15, unit=u.cm, allow_negative=True))
    #print(r)
    df['Luminosity'] = (4*np.pi*(r**2)*(df['Brd_Rate']*flux))
    df = df[df['Luminosity'] > 10**39]
    df = df[ellipse(df['RA'], df['DEC'], df['RA Center'], df['DEC Center'], df['d25'], df['r25'], df['Pos Angle']) <= 1]
    df = df[['Unnamed: 0', 'ID', 'RA', 'DEC', 'Exposure Time', 'Sft_Cts', 'Sft_Rate', 'Med_Cts', 'Med_Rate', 'Hrd_Cts', 'Hrd_Rate', 'Brd_Cts', 'Brd_Rate', 'Luminosity', 'HR1', 'HR2']]
    df.rename(columns = {'Unnamed: 0':'Object', 'ID':'Source ID', 'Sft_Cts':'Soft Counts', 'Sft_Rate':'Soft Count Rate', 'Med_Cts':'Medium Counts', 'Med_Rate':'Medium Count Rate', 'Hrd_Cts':'Hard Counts', 'Hrd_Rate':'Hard Count Rate', 'Brd_Cts':'Broad Counts', 'Brd_Rate':'Broad Count Rate', 'HR1':'Hardness Ratio 1', 'HR2':'Hardness Ratio 2'}, inplace = True)
    frames.append(df)

sources = pd.concat(frames)
sources = sources.reset_index()
del sources['index']

sources

#sources.to_csv('Senior/Honors Thesis/source_list.csv')

0.069500139 0.0166667 236.0
